In [46]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from datetime import datetime, timedelta

Area_row = "B"

Description_row = "C"
# Define the file path and sheet name
file_path = r"C:\Users\grellama\OneDrive - Continental Resources\Personel\weekly updates\project_tracker.xlsx"



# Load the Excel file
workbook = load_workbook(file_path)

# Create an empty dataframe to store the groups
df_groups = pd.DataFrame(columns=["Group", "Content", "SheetsAppended"])

# Get the sheet names in the workbook
sheet_names = workbook.sheetnames

# Iterate through all sheets in the Excel file
for i in range(len(sheet_names) - 1):
    # Get the current and next sheet
    current_sheet = workbook[sheet_names[i]]
    next_sheet = workbook[sheet_names[i + 1]]

    # Iterate through rows starting from the 3rd row
    for row in range(3, current_sheet.max_row + 1):
        # Get the content of the current and next cells in column B
        current_cell = current_sheet[Description_row + str(row)].value
        next_cell = next_sheet[Description_row + str(row)].value

        # Check if both cells have content
        if current_cell and next_cell:
            # Split the content into words
            current_words = set(str(current_cell).lower().split())
            next_words = set(str(next_cell).lower().split())

            # Calculate the similarity score
            similarity_score = len(current_words.intersection(next_words)) / len(current_words)

            # Check if the similarity score is greater than or equal to 0.5 (50%)
            if similarity_score >= 0.5:
                # Get the content of the current row in column B and D
                group_name = current_sheet[Description_row + str(row)].value
                content = current_sheet["D" + str(row)].value

                # Check if the group already exists in the dataframe
                if group_name in df_groups["Group"].values:
                    # Append the content to the existing group on another row
                    df_groups.loc[df_groups["Group"] == group_name, "Content"] += ", " + str(content)
                    
                    df_groups.loc[df_groups["Group"] == group_name, "SheetsAppended"] += 1
                else:
                    # Create a new dataframe for the new group
                    new_group = pd.DataFrame({"Group": [group_name], "Content": [content], "SheetsAppended": [1]})
                    # Concatenate the new group dataframe with the existing groups dataframe
                    df_groups = pd.concat([df_groups, new_group], ignore_index=True)

# Sort the groups by the number of sheets appended in descending order
df_groups = df_groups.sort_values("SheetsAppended", ascending=False)

# set the Group column as the index
df_groups.set_index("Group", inplace=True)

In [47]:
df_groups.head()

,Content,SheetsAppended
Group,,
Spacing Study,• Studied paper SPE-209171-MS to review publ...,21
SplitCAST implementation,"• Continued to review original code, streaml...",10
2023 Goals,"• Added ideas to goals, • Added ideas to g...",4
Professional\nDevelopment/E&P Warehouse Migration,• Worked with IT to troubleshoot workspace acc...,4
GOR Study,• Investigating existing publication in time...,3


In [48]:
df_groups.index

Index(['Spacing Study', 'SplitCAST implementation', '2023 Goals',
       'Professional\nDevelopment/E&P Warehouse Migration', 'GOR Study',
       'Spacing Study (rolling GOR study under this same category)',
       'Professional\nDevelopment/Miscellaneous'],
      dtype='object', name='Group')

In [49]:
df_groups.loc["SplitCAST implementation", "Content"]

'•  Continued  to review original code, streamlined logic and discarded unused code. Documented new versions. Completed, •  Feedback received from Anadarko Asset team. Conducted and led meeting with RD engineers in order to deep dive into SplitCAST methodology.\n•  Team interested in evaluating select group of wells in SCOOP and STACK and review well stats related to number of splits vs. EUR variance.\n•  Currently waiting on well list., •  Reached out to Anadarko RD team to follow up on well list to forecast., •  Reviewed spreadsheet. Working with Ann to calc variance vs actuals on Q422 forecasts\n, Ann finished running economics. Waiting on dataset to calculate variance metrics. Visualize results , Continued coordinating variance for well set B:\n•  Met and reviewed EUR with Ann, will need to re-run economics.\n•  Meeting with Anadarko RD, expect Spotfire project to visualize results for next week., Continued coordinating variance for well set B:\n•  Met and reviewed EUR with Ann, wi

In [50]:
#Retrieve the path for folder in the `file_path` variable
folder_path = file_path[:file_path.rfind("\\")]

#export the dataframe to an Excel file named project_summaries.xlsx, and save it in the folder_path
df_groups.to_excel(folder_path + "\\project_summaries.xlsx")


